In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig, pipeline
import torch
import pandas as pd
from peft import (
    AutoPeftModelForCausalLM,
    get_peft_model,
    LoraConfig,
    PeftConfig,
    PeftModel
)
import re

device = "cuda:0"

- peft 학습한 것 가져오기

In [2]:
peft_model_id = "../result/LDCC-SOLAR-10.7B_2024-03-06_01-30-41/best"
load_tokenizer_name = "../result/LDCC-SOLAR-10.7B_2024-03-06_01-30-41"

In [3]:
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=False,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype="float16",
            )

models = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, revision="v1.1", quantization_config=bnb_config)
models = PeftModel.from_pretrained(models, peft_model_id)
models = models.merge_and_unload() # 7481 -> 8047

tokenizers = AutoTokenizer.from_pretrained(load_tokenizer_name)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:82: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


- 일반 가져오기

In [ ]:
model_id = "LDCC/LDCC-SOLAR-10.7B"
bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=False,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype="float16",
        )
lora_config = LoraConfig(
                lora_alpha=32,
                lora_dropout=0.1,
                r=8,
                bias="none",
                task_type="CAUSAL_LM",
            )
models = AutoModelForCausalLM.from_pretrained(model_id, revision="v1.1", quantization_config=bnb_config)
models.config.use_cache = False
models.config.pretraining_tp = 1
models.enable_input_require_grads()
model = get_peft_model(models, lora_config)

# tokenizers = AutoTokenizer.from_pretrained(model_id)

---

In [4]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test_split_category.csv")

print(tokenizers.pad_token)
print(tokenizers.eos_token)

test.head(5)

</s>
<|im_end|>


,id,질문,category
0,TEST_000,방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?,마감재
1,TEST_000,원목사이딩을 사용하는 것에 어떤 단점이 있을까요?,마감재
2,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?,마감하자
3,TEST_002,큐블럭의 단점을 알려주세요.,마감재
4,TEST_002,압출법 단열판을 사용하는 것의 장점은 무엇인가요?,마감재


In [5]:
category_info = {
    "건축구조": "여러 가지 건축 재료를 이용하여 건축물을 형성하는 일 또는 그 건축물에 관한 질문입니다.",
    "마감재": "건물의 겉면을 마감하는 데 쓰는 재료 및 외부의 여러 가지 영향으로부터 건물을 보호하는 것에 관한 질문입니다.",
    "마감하자": "건물의 겉면을 마감하는 데 쓰는 재료 및 건물 보호 재료에 생기는 문제에 관한 질문입니다.",
    "시공": "공사를 시행하면서 사용하는 재료나 방법에 관한 질문입니다.",
    "인테리어": "실내를 장식하는 일이나 실내 장식용품에 관한 질문입니다.",
    "타 마감하자": "표면에 물방울이 맺혀 문제가 생기는 결로 등 생활하면서 생기는 문제에 관한 질문입니다.",
    "기타": "집 내부와 생활 기준 및 건축의 포괄적인 분야에 관한 질문입니다.",
}

In [6]:
id = 5

input_text = test.loc[id, '질문']
category = test.loc[id, 'category']

with open("../template/ldcc.txt", "r", encoding="utf-8") as file:
    content = file.read()

answer_start_index = content.find("<answer>")
content = content[:answer_start_index]
content = content.replace("<question>", input_text.strip().replace('"', ""))
content = content.replace("<category>", category_info.get(category))

input_ids = tokenizers.encode(
            content, padding=False, max_length=256, return_tensors="pt", add_special_tokens=False
        )
print(content, "\n--------------\n")
print(content.replace("\n", "\\n"), "\n--------------\n")
print(input_ids)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


여러 가지 건축 재료를 이용하여 건축물을 형성하는 일 또는 그 건축물에 관한 질문입니다.
Q: 철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?
A: 
--------------

여러 가지 건축 재료를 이용하여 건축물을 형성하는 일 또는 그 건축물에 관한 질문입니다.\nQ: 철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?\nA: 
--------------

tensor([[28705, 29827, 30244, 28705, 29135, 29161, 28705, 30771, 46620, 28705,
         30014, 30221, 29200, 28705, 29015, 29424, 29136, 29827, 28705, 30771,
         46620, 30895, 29189, 28705, 30498, 29465, 29136, 29175, 28705, 29415,
         28705, 31292, 29175, 28705, 29614, 28705, 30771, 46620, 30895, 29148,
         28705, 30224, 29282, 28705, 31959, 29710, 29429, 29194, 29043, 28723,
            13, 28824, 28747, 28705, 46627, 46674, 29779, 29841, 29200, 28705,
         29315, 29424, 29136, 29175, 28705, 29511, 46710, 28705, 30771, 30895,
         29148, 29305, 28725, 28705, 30315, 30243, 28705, 31481, 29844, 29200,
         28705, 46651, 29015, 29164, 28705, 29744, 29282, 28705, 29236, 30010,
         28705, 30240, 30979, 29538, 28

In [7]:
output_sequences = models.generate(
        input_ids=input_ids.to("cuda"),
        max_length=512,
        temperature=0.2,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )

print(tokenizers.decode(output_sequences[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


여러 가지 건축 재료를 이용하여 건축물을 형성하는 일 또는 그 건축물에 관한 질문입니다.
Q: 철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?
A:철골구조의 경우, 단열 효과를 높이기 위해 벽체와 천장 등의 내부 마감재를 단열재로 채워주는 것이 일반적인 방식입니다. 또한 외피 단열막을 설치하여 빌딩 표면 temperature to zero'를 유지하는 것도 중요합니다. 이렇게 함으로써 단열재가 건물 내부의 온도를 안정화시키고, 열이 밖으로 누출되는 것을 방지할 수 있습니다.<|im_end|>


In [ ]:
mm = "철골구조의 경우, 단열 효과를 높이기 위해 벽체와 천장간의 거리를 늘리고 두께를 증대하는 것이 좋습니다. 또한 외단열 시스템을 도입하여 빌딩 겉면으로의 열전달을 차단하는 방법이 유용합니다. 이를 통해 단열 효과를 향상시키고 건물 내부의 온도를 안정화할 수 있습니다.<|im_end|>"
nn = "철골구조의 경우, 단열 효과를 높이기 위해 벽체와 천장 등의 내부 마감재를 단열재로 채워주는 것이 일반적인 방식입니다. 또한 외피 단열막을 설치하여 빌딩 표면 temperature to zero'를 유지하는 것도 중요합니다. 이렇게 함으로써 단열재가 건물 내부의 온도를 안정화시키고, 열이 밖으로 누출되는 것을 방지할 수 있습니다.<|im_end|>"

---

# similarity

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
model = SentenceTransformer("distiluse-base-multilingual-cased-v1")

In [ ]:
k = model.encode(["방청 페인트는 다양한 종류로 분류됩니다 . 그 중에서 가장 흔 히 사용되는 것은 광 명단 페인 트, 방청산화철 페인 트, 알루미늄 페인 트, 역청질 페인 트, 워시 프라이머 , 크롬 산아연 페인 트 등이 있습니다 . 이러한 다양한 종류 의 방청 페인트가 각자의 특성과 용도에 맞 게 사용됩니다 ."])
t = model.encode(["방청 페인트는 다양한 종류로 분류됩니다. 그 중에서 가장 흔히 사용되는 것은 광명단 페인트, 방청산화철 페인트, 알루미늄 페인트, 역청질 페인트, 워시 프라이머, 크롬 산아연 페인트 등이 있습니다. 이러한 다양한 종류의 방청 페인트가 각자의 특성과 용도에 맞게 사용됩니다."])
cosine_similarity(k, t)

---

# start token이 잘못되어 보정

In [ ]:
import json
import pandas as pd
import re
from sentence_transformers import SentenceTransformer

In [ ]:
def submission(CFG, preds):
    model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
    nl = pd.read_csv(f"../{CFG['DATA_PATH']}/{CFG['TEST_DATA']}")
    submit = pd.read_csv(f"../{CFG['DATA_PATH']}/{CFG['SUBMISSION_DATA']}")
    submission_name = "DataVortexS-10.7B-dpo-v1.11_2024-03-04_02-11-13"
    nl["답변"] = preds
    nl.to_csv(f'../{CFG["SAVE_PATH"]}/{submission_name}/NL_{submission_name}.csv', index=False)
    if len(nl) != len(submit):
        nl = (
            nl.groupby("id")["답변"]
            .apply(lambda x: " ".join(x.astype(str)))
            .reset_index()
        )
        preds = nl["답변"]
        nl.to_csv(f'../{CFG["SAVE_PATH"]}/{submission_name}/NL_merge_{submission_name}.csv', index=False)
    pred_embeddings = model.encode(preds)
    print("Shape of Prediction Embeddings: ", pred_embeddings.shape)
    submit.iloc[:, 1:] = pred_embeddings
    submit.to_csv(f'../{CFG["SAVE_PATH"]}/{submission_name}/{submission_name}.csv', index=False)

In [ ]:
with open(f'../config/datavortex_0304.json', 'r', encoding='utf-8') as file:
    CFG = json.load(file)
    
infer_csv = pd.read_csv("../result/NL_DataVortexS-10.7B-dpo-v1.11_2024-03-04_02-11-13.csv")

In [ ]:
preds = []
start_token = "### Assistant: "
for full_text in infer_csv['답변']:
    answer_start = full_text.find(start_token) + len(start_token)
    answer_only = full_text[answer_start:].strip()
    answer_only = answer_only.replace("\n", "")
    answer_only = re.sub(r'\s+', ' ', answer_only)
    answer_only = answer_only.replace("<|im_end|>", "")
    answer_only = answer_only.replace("</s>", "")
    preds.append(answer_only)

In [ ]:
submission(CFG, preds)